# DCDB

In [33]:
import pandas as pd
import numpy as np
import os 

import sklearn.metrics.pairwise
import matplotlib.pyplot as plt
from tdc.chem_utils import MolConvert

import pickle
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import nbimporter
from assistant import distinct, drug_index, save, labels_to_tensor, vectorize_smile, drug_combi_index, data_indexed_combi, vectorize_drugcombi

## DCDB Dataset
    [1] Drug Combination - Drug Components 
    [2] Drug Combination - Drug Combination Usages
    [3] Drug Combination Usages - Diesase 

### Preprocessing 

In [6]:
# [1] Drug Combination to Components
drug_comb = pd.read_csv("./data/DCDB/drugCombinations.txt", delimiter = "\t")

,DrugCombination_ID,Components_Name,Componets_ID
0,DC000348,Bismuth Subsalicylate; Metronidazole; Tetracyc...,DCC0187/DCC0235/DCC0338
1,DC000349,Brimonidine; Timolol,DCC0072/DCC0106
2,DC000350,Betamethasone; Calcipotriol,DCC0095/DCC0358
3,DC000351,Betamethasone; Clotrimazole,DCC0033/DCC0095
4,DC000352,Cerulenin; Levodopa,DCC0274/DCC0326
...,...,...,...
1358,DC007307,Phenazopyridine hydrochloride; Sulfamethoxazole,DCC0266/DCC1844
1359,DC007308,Phenazopyridine hydrochloride; Sulfamethoxazol...,DCC0093/DCC0266/DCC1844
1360,DC007309,Phenazopyridine hydrochloride; Sulfisoxazole,DCC0035/DCC1844
1361,DC007310,Phenylephrine hydrochloride; Pyrilamine maleate,DCC0075/DCC1849


As you can see, some drug combinations have more than two drugs. Since we only consider pair-wise dataset, we filtered out the datasets to be left with only two drugs are used.

In [7]:
# seperate compnents to individuals
drug_comb["comp1"] = drug_comb.Componets_ID.str.split('/').str[0].fillna(0)
drug_comb["comp2"] = drug_comb.Componets_ID.str.split('/').str[1].fillna(0)
drug_comb["comp3"] = drug_comb.Componets_ID.str.split('/').str[2].fillna(0)

# select the data (rows) that only two drugs are used
drug_comb = drug_comb[drug_comb.comp3==0].iloc[:, : -1]
drug_comb = drug_comb.reset_index(drop=True)

In [8]:
# [2] Drug Combination to Usage 
dc_to_dcu = pd.read_csv("./data/DCDB/dcdb/DC_TO_DCU.txt", delimiter="\t")

,DC_ID,DCU_ID
0,DC000001,DCU00331
1,DC000002,DCU00330
2,DC000003,DCU00326
3,DC000004,DCU00327
4,DC000005,DCU00222
...,...,...
1788,DC004864,DCU07059
1789,DC004872,DCU07070
1790,DC004873,DCU07071
1791,DC004887,DCU07094


In [9]:
# [3] Drug Usages to Disease
drug_usage = pd.read_csv("./data/DCDB/dcdb/DC_USAGE.txt", delimiter="\t")

,DCU_ID,DOSE,ADMINISTRATION,DISEASE,EFFICACY,EFFECT_TYPE,STAGE,SOURCE,ICD10_S,ICD10_L,EFFECTS_T,TOXICITY,OVERALL
0,DCU01786,Didanosine: 250 mg; Lopinavir: 400 mg; Nevirap...,NaN,HIV Infections,Efficacious,Unclear,Phase 2,NCT00109590,B20 Human immunodeficiency virus [HIV] disease,A00-B99 Certain infectious and parasitic dise...,"At entry, the 169 participants had a median CD...",NaN,1 month of dual therapy after SD-NVP prevents ...
1,DCU01796,Dasatinib: 100 mg; Docetaxel: 75 mg/m2; Predni...,NaN,Prostatic Neoplasms,Non-efficacious,Unclear,Phase 3,NCT00744497,C61 Malignant neoplasm of prostate,C00-D48 Neoplasms; C60-C63 Malignant neoplas...,"At final analysis, median follow-up was 19.0 m...",The most common grade 3-4 adverse events inclu...,no improvement on overall survival
2,DCU01798,Bevacizumab: 15 mg/kg; Capecitabine: 825 mg/m2...,NaN,Breast Cancer,Efficacious,Unclear,Phase 3,NCT00408408,C50 Malignant neoplasms of breast,C00-D48 Neoplasms; C50-C50 Malignant neoplas...,The addition of bevacizumab significantly incr...,Both capecitabine and gemcitabine were associa...,significantly increased the rate of pathologic...
3,DCU01800,Fluticasone Propionate: 100 mcg; Montelukast: ...,NaN,Asthma,Need further study,Unclear,Phase 3,NCT00395304,J45 Asthma,J00-J99 Diseases of the respiratory system; ...,The response to long-acting beta-agonist twice...,NaN,significantly more likely to provide the best ...
4,DCU01843,Cyclosporine: 300 mg; Mycophenolate mofeti: 2 ...,NaN,Kidney Transplantation,Need further study,Unclear,Phase 4,NCT00812123,N18.6 End stage renal disease,N00-N99 Diseases of the genitourinary system...,NaN,"Kidney function was similar in both groups, wh...","compared with sirolimus regimen, with lower tu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1808,DCU00368,Hydrocortisone Acetate: 1%; Neomycin Sulfate: ...,Cream; Topical,Ear or eye infections,Efficacious,NaN,Approved,FDA orange book,H00-H95,H00-H95,NaN,NaN,NaN
1809,DCU00369,Hydrocortisone Acetate: 1.5%; Oxytetracycline ...,Suspension/Drops; Ophthalmic,Used in inflammatory conditions of the palpebr...,Efficacious,NaN,Approved,FDA orange book,H10-H11 Disorders of conjunctiva; H15-H22 Di...,H00-H59 Diseases of the eye and adnexa; H10-H...,NaN,NaN,NaN
1810,DCU00370,Hydrocortisone Acetate: 1%; Pramoxine Hydrochl...,"Aerosol, Metered; Topical","Minor skin irritations, anal itching, hemorrho...",Efficacious,NaN,Approved,FDA orange book,"L29.0 Pruritus ani; R52 Pain, unspecified",L00-L99 Diseases of the skin and subcutaneou...,NaN,NaN,NaN
1811,DCU10175,Docetaxel: 75 mg/m2; Epirubicin: 75 mg/m2; Tra...,NaN,Breast Neoplasms,Efficacious,Unclear,Phase 2,NCT00242203,C50 Malignant neoplasms of breast,C00-D48 Neoplasms; C50-C50 Malignant neoplas...,NaN,NaN,NaN


In [10]:
# call in identifier for each drug
identifier = pd.read_csv("./data/DCDB/components_identifier.txt", delimiter="\t")  # 876 - but only 759 drugs are used for out research

# Cas_Number is used as identifier 
drug_cas = identifier.dropna(subset=["CAS_Number"]).drop_duplicates(subset=["CAS_Number"])  # 863

# preprocess for further web-site crawling 
drug_cas["cas_number"] = pd.DataFrame(drug_cas.CAS_Number.str.split(':', 1, expand=True)[1])
drug_cas = drug_cas[["DCC_ID","cas_number"]]
drug_cas

,DCC_ID,cas_number
0,DCC1838,135-07-9
1,DCC0413,194804-75-6
2,DCC0520,107-43-7
3,DCC0639,173937-91-2
4,DCC0029,75330-75-5
...,...,...
871,DCC0239,89365-50-4
872,DCC0148,152459-95-5
873,DCC0149,124-94-7
874,DCC0150,154598-52-4


In [11]:
# data_full
data_full = drug_usage.merge(dc_to_dcu).dropna(subset=["ICD10_L"])
data_full = data_full.merge(drug_comb, left_on = "DC_ID", right_on="DrugCombination_ID")
data_full = data_full[["DC_ID", "comp1", "comp2","DISEASE", "ICD10_L"]]

data_full = data_full.drop_duplicates().reset_index(drop=True)
data_full

,DC_ID,comp1,comp2,DISEASE,ICD10_L
0,DC005362,DCC0306,DCC1524,Major Depressive Disorder,F00-F99 Mental and behavioural disorders; F30...
1,DC001170,DCC0145,DCC0247,Diabetes Mellitus,"E00-E90 Endocrine, nutritional and metabolic ..."
2,DC004166,DCC0245,DCC0532,Advanced Cancers,C Malignant neoplasm
3,DC004170,DCC0134,DCC0169,Marfan Syndrome,"Q00-Q99 Congenital malformations, deformatio..."
4,DC004318,DCC0044,DCC1247,Postoperative Ileus,K00-K93 Diseases of the digestive system; K5...
...,...,...,...,...,...
1013,DC000056,DCC0047,DCC0060,For patients with hypertension,I00-I99 Diseases of the circulatory system; I...
1014,DC000075,DCC0124,DCC0184,Ear infections,H00-H59 Diseases of the eye and adnexa
1015,DC000074,DCC0124,DCC0325,Ear infections,H00-H59 Diseases of the eye and adnexa
1016,DC000077,DCC0143,DCC0184,Used in inflammatory conditions of the palpebr...,H00-H59 Diseases of the eye and adnexa; H10-H...


In [12]:
# seperate above redundant data into lists with only one ICD10 term stack = [] 
for i in data_full.index:  
    codes = data_full.iloc[i].ICD10_L.split('| ')
    if len(codes) > 1 :
        stack.append(data_full.iloc[i])
        
stacked = pd.DataFrame(stack)
new = []
for i in range(len(stacked)):
    codes = stacked.ICD10_L.iloc[i].split(' | ')
    for j,code in enumerate(codes):
        new_data = stacked.iloc[i]
        new_data[-1] = code 
        
        new.append(list(new_data))
        
news = pd.DataFrame(new)
news.columns = data_full.columns

# reconstruct data_full
data_full = data_full.drop(stacked.index).reset_index(drop=True)
data_full = pd.concat([data_full,news])

#### Crawling SMILES string
SMILES strings are crawled using Cas-number via in "https://commonchemistry.cas.org"

In [ ]:
def getSMILE(identifier):
    """
    Crawl SMILE data using CAS-number automatically via chrome Driver
    Input: Cas-Number
    output: SMILE string for input Cas-number
    """
    # Connect to Chrome Driver 
    driver = webdriver.Chrome("/usr/local/bin/chromedriver")
    driver.get(url)
    
    # Automated typing your identifier
    upload = driver.find_element_by_xpath("/html/body/app-root/app-landing/div/div/div[2]/app-omni-bar/form/div/div/div/input")
    upload.click()
    upload.send_keys(identifier)
    time.sleep(1)
    
    # click the submit button
    sub = driver.find_element_by_css_selector("body > app-root > app-landing > div > div > div.omni-bar > app-omni-bar > form > div > div > div > button")
    sub.send_keys(Keys.ENTER)
    time.sleep(2)
    
    try:
        result = driver.find_element_by_class_name("result-rn")
        result.click()
        time.sleep(2)
    
        # get SMILES 
        smile = driver.find_element_by_xpath("/html/body/app-root/app-detail/div/div/div/div[3]/detail-info/div/p[3]").text
        driver.quit()
    except:
        return("not exist")
    return(smile)

In [ ]:
# CAS website
url = "https://commonchemistry.cas.org/"
html = requests.get(url)

In [ ]:
smiles = drug_cas.cas_number
DCDB_smile = []
for i in smiles:
    if i != i:
        DCDB_smile.append([i,"not exist"])
    else:
        singleSMILE = getSMILE(i)
        DCDB_smile.append([i,singleSMILE])

# DCDB drug dataset with SMILES
with open("./data/DCDB/DCDB_smile", "wb") as fp:
    pickle.dump(DCDB_smile, fp)

In [16]:
with open("./data/DCDB/DCDB_smile" , "rb") as fp:
    DCDB_smile = pickle.load(fp)

In [17]:
# distinct Drug with SMILES string
Drug_smile = pd.DataFrame(DCDB_smile, columns=["cas_number", "SMILE"])    # 864
Drug_smile = Drug_smile[Drug_smile.SMILE != "not exist"]   # filter out 94 with "not exist" --> 770
Drug_smile = Drug_smile[Drug_smile.SMILE != ""].drop_duplicates()

,cas_number,SMILE
0,68-41-7,N[C@H]1C(=O)NOC1
1,144689-63-4,C(N1C(C(OCC2=C(C)OC(=O)O2)=O)=C(C(C)(C)O)N=C1C...
2,58-55-9,O=C1C2=C(N(C)C(=O)N1C)N=CN2
3,137-58-6,N(C(CN(CC)CC)=O)C1=C(C)C=CC=C1C
4,15686-51-8,[C@@](OCC[C@@H]1N(C)CCC1)(C)(C2=CC=C(Cl)C=C2)C...
...,...,...
858,59122-46-2,C(=C/CC(CCCC)(C)O)\[C@@H]1[C@@H](CCCCCCC(OC)=O...
859,89365-50-4,C(CNCCCCCCOCCCCC1=CC=CC=C1)(O)C2=CC(CO)=C(O)C=C2
860,152459-95-5,N(C=1N=C(C=CN1)C=2C=CC=NC2)C3=CC(NC(=O)C4=CC=C...
861,124-94-7,F[C@@]12[C@]([C@]3([C@](C)(C[C@@H]1O)[C@](C(CO...


In [18]:
# reconstruct drug data with morgan fingerprint applicabile
drug_morgan = Drug_smile.merge(drug_cas) 

# filter out data which occur error in converting SMILE string into morgan fingerprint
SMILE_nontypes = ["DCC0117", "DCC0013", "DCC0245", "DCC0120", "DCC0054"]
drug_morgan = drug_morgan.drop(drug_morgan.loc[drug_morgan.DCC_ID.isin(SMILE_nontypes)].index).reset_index(drop=True)

In [23]:
# data_full
data_full = data_full.merge(drug_morgan, left_on = "comp1", right_on="DCC_ID")
data_full = data_full.merge(drug_morgan, left_on = "comp2", right_on="DCC_ID")

# data_full with SMILE string but also applicable for morgan fingerprint
data_full = data_full[["DC_ID", "comp1", "comp2", "DISEASE", "SMILE_x" ,"SMILE_y","ICD10_L"]].rename(columns={"SMILE_x":"comp1_SMILE", "SMILE_y":"comp2_SMILE"})
data_full

,DC_ID,comp1,comp2,DISEASE,comp1_SMILE,comp2_SMILE,ICD10_L
0,DC005362,DCC0306,DCC1524,Major Depressive Disorder,C(C(NC(C)(C)C)C)(=O)C1=CC(Cl)=CC=C1,C(CCN(C)C)[C@@]1(C=2C(CO1)=CC(C#N)=CC2)C3=CC=C...,F00-F99 Mental and behavioural disorders; F30...
1,DC006430,DCC0306,DCC1524,Major Depressive Disorder,C(C(NC(C)(C)C)C)(=O)C1=CC(Cl)=CC=C1,C(CCN(C)C)[C@@]1(C=2C(CO1)=CC(C#N)=CC2)C3=CC=C...,F00-F99 Mental and behavioural disorders; F30...
2,DC003707,DCC1293,DCC1524,Major Depressive Disorder,O([C@@H](CCNC)C1=CC=CS1)C=2C3=C(C=CC2)C=CC=C3.Cl,C(CCN(C)C)[C@@]1(C=2C(CO1)=CC(C#N)=CC2)C3=CC=C...,F00-F99 Mental and behavioural disorders; F30...
3,DC004002,DCC1345,DCC1524,Major Depressive Disorder,C(CNC(C)=O)CS(=O)(=O)O,C(CCN(C)C)[C@@]1(C=2C(CO1)=CC(C#N)=CC2)C3=CC=C...,F00-F99 Mental and behavioural disorders; F30...
4,DC001170,DCC0145,DCC0247,Diabetes Mellitus,O(CC(CNC(C)C)O)C1=CC=C(COCCOC(C)C)C=C1,C(N1C=2C(N=C1CCC)=C(C)C=C(C2)C=3N(C)C=4C(N3)=C...,"E00-E90 Endocrine, nutritional and metabolic ..."
...,...,...,...,...,...,...,...
765,DC000047,DCC0119,DCC0165,For patients with hypertension,C([C@@H](N[C@@H](CCC1=CC=CC=C1)C(OCC)=O)C)(=O)...,C(CCCN(CCC1=CC(OC)=C(OC)C=C1)C)(C(C)C)(C#N)C2=...,I00-I99 Diseases of the circulatory system; I...
766,DC003251,DCC1242,DCC1314,Lymphoma,CC1(C)C=2C(C(C)(C)CC1)=CC(C)=C(C(=C)C3=CC=C(C(...,O(C)C=1C2=C(C=C3C1OC=C3)C=CC(=O)O2,C00-D48 Neoplasms; C81-C96 Malignant neoplas...
767,DC003251,DCC1242,DCC1314,Lymphoma,CC1(C)C=2C(C(C)(C)CC1)=CC(C)=C(C(=C)C3=CC=C(C(...,O(C)C=1C2=C(C=C3C1OC=C3)C=CC(=O)O2,C00-D48 Neoplasms; C81-C96 Malignant neoplas...
768,DC007307,DCC0266,DCC1844,"For the treatment of bronchitis, prostatitis a...",S(NC=1C=C(C)ON1)(=O)(=O)C2=CC=C(N)C=C2,N(=NC1=CC=CC=C1)C2=C(N)N=C(N)C=C2.Cl,J00-J99 Diseases of the respiratory system; ...


### Indexing Dataset

In [27]:
# drug index
drug_idx = pd.DataFrame(set(data_full.comp1).union(set(data_full.comp2)), columns=["DCC_ID"]) 
drug_idx["Drug_Index"] = drug_idx.index   # 546
 
# ICD10 codes index
ICD10_idx = pd.DataFrame(data_full.ICD10_L.unique(), columns=["ICD10_L"])
ICD10_idx["ICD10_Index"] = ICD10_idx.index

In [29]:
# indexing full_data with indexed Drug and Disease
data_indexed = data_full.merge(drug_idx, left_on="comp1", right_on="DCC_ID").rename(columns={"Drug_Index":"Drug1_Index"})
data_indexed = data_indexed.merge(drug_idx, left_on="comp2", right_on="DCC_ID").rename(columns={"Drug_Index":"Drug2_Index"})
data_indexed = data_indexed.merge(ICD10_idx, on="ICD10_L").drop(columns=["DCC_ID_x", "DCC_ID_y"])

## Labels

In [31]:
# labels triplets [Drug1, Drug2, Disease]
labels_triplets = np.array(data_indexed[['Drug1_Index', 'Drug2_Index', 'ICD10_Index']]).astype(int)

In [32]:
# label tensor (Yabc)
labels_tensor = labels_to_tensor(labels_triplets, drug_idx["Drug_Index"], ICD10_idx["ICD10_Index"])

(546, 546, 268)

## Feature Vectorizatoin

### Drug
[1] SMILES

In [ ]:
# merge SMILES string to drug index dataset
drug_idx_smile = drug_idx.merge(drug_morgan)

In [ ]:
# vectorize with one-hot coding of SMILES string
drug_vectorized_smile = vectorize_smile(drug_idx_smile.SMILE)  # (546, 221, 36)

# reshape into 2D dimension  
drug_vectorized_smile = drug_vectorized_smile.reshape(drug_vectorized_smile.shape[0], drug_vectorized_smile.shape[1] * drug_vectorized_smile.shape[2])   # (546, 7956)

In [ ]:
# read the data inside the xml file (download from ICD10) to a variable under the name data
with open('Tabular.xml', 'r') as f:   
    data = f.read() 

# use beautifulsoup parser 
bs_data = BeautifulSoup(data, 'xml') 
codes = bs_data.find_all('chapter')  

[2] Morgan Fingerprint

The molecule conversion API of TDC is applied.

In [ ]:
# convert SMILES string into morgan fingerprint with 1024 features 
converter = MolConvert(src = 'SMILES', dst = 'Morgan')
drug_vectorized_morgan = converter(drug_idx_smile.SMILE.values.tolist()).astype("int8")  # (546, 1024) 

### Disease (ICD10 Code)

#### [1] drug pair labels as feature

In [ ]:
# Indexing Drug-Drug combination (here, only combination matters not the order)
combi_idx = drug_combi_index(drug_idx)  # (207690, 3)

# labels_triplets with extra column of drugcombination index
data_indexed_comb = data_indexed_combi(data_indexed, combi_idx)
labels_triplets_drugcombi = np.array(data_indexed_comb[['Drug1_Index', 'Drug2_Index', 'ICD10_Index', "Combi_Index"]]).astype(int)

In [ ]:
# Vectorizing into 2D tensor of data_indexed 
ICD10_vectorized = vectorize_drugcombi(labels_triplets_drugcombi, combi_idx, ICD10_idx)

#### [2] ICD10 codes hierarchy as feature
For the research, the ICD10 version 2014 are used since DCDB used that version. The used xml was downloaded from https://www.cms.gov/Medicare/Coding/ICD10/2014-ICD-10-CM-and-GEMs.

In [ ]:
# read the data inside the xml file to a variable under the name data
with open('Tabular.xml', 'r') as f:   
    data = f.read() 

# use beautifulsoup parser 
bs_data = BeautifulSoup(data, 'xml') 
codes = bs_data.find_all('chapter')  

In [ ]:
# build lists of ICD10 terms within levels
levels, level2_list, level3_list, level4_list, level5_list = [], [], [], [], []

for code in codes:
    # Level 1
    level1 = code.find('desc').text.split(' ')[-1].replace('(','').replace(')','')   
    levels.append(level1)

    ## Level 2 
    level2 = code.find_all('sectionRef')
    for i in level2:
        level2_list.append(i.get('id'))

    ### Level 3 & 4 & 5 
    level3 = code.find_all('section')
    for i in level3:
        names = i.find_all('name')
        for j in names:
            if len(j.text) == 3:
                level3_list.append(j.text)
            elif len(j.text) == 5:
                level4_list.append(j.text)    
            elif len(j.text) >= 6:
                level5_list.append(j.text)

#### Preprocessing (Redundant codes are removed) 
- T20-T32 Burns and corrosions
- T20-T25 Burns and corrosions of external body surface, specified by site
- T26-T28 Burns and corrosions confined to eye and internal organs
- T30-T32 Burns and corrosions of multiple and unspecified body regions

--> Filter out 'T20-T32'

- Y62-Y84 Complications of medical and surgical care
- Y62-Y69 Misadventures to patients during surgical and medical care
- Y70-Y82 Medical devices associated with adverse incidents in diagnostic and therapeutic use
- Y83-Y84 Surgical and other medical procedures as the cause of abnormal reaction of the patient, or of later complication, without mention of misadventure at the time of the procedure

--> Filter out 'Y62-Y84'

- V00-X58 Accidents
- V00-V99 Transport accidents
- V00-V09 Pedestrian injured in transport accident
...
- W00-X58 Other external causes of accidental injury
- W00-W19 Slipping, tripping, stumbling and falls

--> Filter out 'V00-X58', 'V00-V99', 'W00-X58'

In [ ]:
level2_list.remove('T20-T32')
level2_list.remove('Y62-Y84')
level2_list.remove('V00-X58')
level2_list.remove('V00-V99')
level2_list.remove('W00-X58')

In [ ]:
# Seperate the lists into subsets 

# Level 2
last = [] 
for i in levels:
    first_last = i.split('-')
    last.append(first_last[-1])
last = list(last)

part, level2_part = [], []
for level2 in level2_list: 
    part.append(level2)
    level2_last = level2.split('-')[-1]
    if level2_last == last[0]:
        level2_part.append(part)
        part = []
        last.pop(0)
        
# Level 3
last = [] 
for i in level2_list:
    if len(i) > 3:
        last_level= i.split('-')[-1]
    else:
        last_level = i
    last.append(last_level)
last = list(last)

part, level3_part = [], []
for level3 in level3_list: 
    part.append(level3)
    if level3 == last[0]:
        level3_part.append(part)
        part = []
        last.pop(0)

# Level 4
head = [] 
for i in level3_list:
    head.append(i)
head = list(head)

part, level4_part = [], []
for level4 in level4_list: 
    level4_head = level4.split('.')[0]

    if level4_head == head[0]:
        part.append(level4)
        
    elif level4_head != head[0] and len(part) > 0:
        level4_part.append(part)
        head.pop(0)
        part = []
        
    elif len(part) == 0 :
        level4_part.append(None)
        head.pop(0)

# Last one 
level4_part.append(['Z99.1', 'Z99.2', 'Z99.3', 'Z99.8'])

In [ ]:
# build dictionary with all the codes with hierarchy information
level_34 = []
for i, level3_section in enumerate(level3_part):
    levels_dic = {}
    for j, level3 in enumerate(level3_section):
        levels_dic[level3] = level4_part.pop(0)
    level_34.append(levels_dic)

    level_234 = []
for i, level2_section in enumerate(level2_part):
    levels_dic = {}
    for j, level2 in enumerate(level2_section):
        levels_dic[level2] = level_34.pop(0)
    level_234.append(levels_dic)
        
level_1234 = []
levels_dic = {}
for i, level1 in enumerate(levels):
    levels_dic[level1] = level_234.pop(0)    

In [ ]:
# dictionary for each connected levels 
level12_dic, level23_dic, level34_dic = {}, {}, {}

for keys1, values2 in levels_dic.items():
    level12_dic[keys1] = [k for k in values2.keys()]
    for keys2, values3 in values2.items():
        level23_dic[keys2] = [k for k in values3.keys()]
        level34_dic.update(values3)

#### Compute similarity using hierarchy levels

In [ ]:
def node_depth(node):
    """
    Output -> the level or depth of the input nodes in hiearchy ICD10 terms
    
    ### Examples ###
    level 1 : 'A00-B99'
    level 2 : 'A00-A09'
    level 3 : 'A00'
    level 4 : 'A00.1'
    """
    assert len(node) in [3,5,7], "Check your ICD10 code. It should be in level 1(ex,'A00-B99') to level 4(ex,'A00.0')."
    
    if node in levels:  
        return  1
    elif node in level2_list:
        return 2
    elif len(node) == 3:
        return 3
    elif len(node) == 5:
        return 4

In [ ]:
def children_to_parent(node):
    """
    Input: child node
    Output -> Edge connecting child node to its parent node
    """
    
    depth = node_depth(node)

    if depth == 1:
        return((node, 'ICD10'))
    
    if depth == 2:
        for level1, level2_all in level12_dic.items():
            for level2 in level2_all:
                if node == level2:
                     return((level2, level1))
                    
    if depth == 3:
        for level2, level3_all in level23_dic.items():
            for level3 in level3_all:
                if node == level3:
                    return((level3, level2))
    
    if depth == 4:
        for level3, level4_all in level34_dic.items():     
            try:
                for level4 in level4_all:
                    if node == level4:
                        return((level4, level3))
            except:
                pass

In [ ]:
def hierarcy_similarity(node1, node2):
    """
    Input: Two nodes 
    # Calculate hierarcy distance using Adjusted path-based similarity
    # l(vi,vj)=l(vi,lcaij)+l(vj,lcaij)/(1 + l(lcaijt))  ## from Gleb Sologub. On Measuring of Similarity Between Tree Nodes
    # similarity = 1/1(+distance)
    
    Output -> Similarity of two input nodes 
    """
    paths = [] 
    go_paths, down_paths = [], []
    
    if node1 == node2:
        return 1
    
    # depths of initial nodes
    dep_node1 = node_depth(node1)
    dep_node2 = node_depth(node2)
    

    if dep_node1 >= dep_node2:
        node_1, node_2 = node1, node2
    else:
        node_1,node_2 = node2, node1
        
    # start nodes
    start_edge = children_to_parent(node_1)
    end_edge = children_to_parent(node_2)

    go_paths.append(start_edge)
    down_paths.append(end_edge)
    
    # find the paths from each node to root
    # paths from node_1 to troot
    for i in range(node_depth(node_1)):
        new_edge = children_to_parent(go_paths[i][1])
        go_paths.append(new_edge)
        if go_paths[-1][-1] == 'ICD10':
            break
    
    # paths from node_2 to root
    for i in range(node_depth(node_1)):
        new_edge = children_to_parent(down_paths[i][1])
        down_paths.append(new_edge)
        if down_paths[-1][-1] == 'ICD10':
            break
            
    paths.extend(go_paths)
    paths.extend(down_paths)
    
    paths_to_root = {i:paths.count(i) for i in paths}
    final_paths = [key for key,val in paths_to_root.items() if val != 2]
    
    final_nodes = [node for nodes in final_paths for node in nodes]
    final_nodes_levels = [node_depth(node) for node in final_nodes]
    
    adjusted_path_distance = len(final_paths)/(1 + min(final_nodes_levels))
    similarity = 1/(1+adjusted_path_distance)

    return similarity

In [ ]:
# adjust the codes to be comparable with direct ICD10 codes 
adjusted_codes = [] 
for ele in ICD10_last_codes:  
    if len(ele) != 6:
        adjusted_codes.append(ele)
    elif len(ele) == 6:
        adjusted_codes.append(ele[:-1])

In [ ]:
Deep_code = pd.DataFrame(adjusted_codes, columns=["Deep_code"])

# add Deep_code column to ICD10_idx
ICD10_idx = pd.concat([ICD10_idx, Deep_code], axis=1)

## Similarity Kernels

### Drug
[1] SMILES

In [ ]:
# cosine similarity 
K_drug_smile_cos = sklearn.metrics.pairwise.cosine_similarity(drug_vectorized_smile)

In [ ]:
# tanimoto similarity  
K_drug_smile_tanimoto = tanimoto_similarity_matrix(drug_vectorized_smile)

[2] Morgan fingerprint

In [ ]:
# cosine similarity 
K_drug_morgan_cos = sklearn.metrics.pairwise.cosine_similarity(drug_vectorized_morgan)

In [ ]:
# tanimoto similarity
K_drug_morgan_tanimoto = tanimoto_similarity_matrix(drug_vectorized_morgan)

### Disease

[1] drug pair labels as feature

In [ ]:
# cosine similarity
K_ICD10_pair_cos = sklearn.metrics.pairwise.cosine_similarity(ICD10_vectorized)

In [ ]:
# jaccard similarity
K_ICD10_pair_jacc = 1 - sklearn.metrics.pairwise_distances(ICD10_vectorized, metric ='jaccard')   

[2] Using ICD10 codes as feature

In [ ]:
# hierarchy similarity
ICD10_hier_sim = np.zeros((len(ICD10_idx), len(ICD10_idx)))

for i, ICD10_1 in enumerate(ICD10_idx.Deep_code):
    for j, ICD10_2 in enumerate(ICD10_idx.Deep_code):
        try:
            ICD10_hier_sim[i,j] = hierarcy_similarity(ICD10_1, ICD10_2)
        except:
            pass

In [ ]:
# jaccard similarity for text 
def Jaccard_Similarity(doc1, doc2): 
    """
    Input : two strings to be compare 
    Output : jaccard similarity score 
    """
    # list the distinct words in a document 
    try:
        doc1, doc2 = doc1.replace(";",""), doc2.replace(";","")
    except:
        pass
    words_doc1, words_doc2 = set(doc1.lower().split()), set(doc2.lower().split())

    # find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)
    
    # find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)

    # calculate jaccard similarity score using length of intersection set divided by length of union set
    return(float(len(intersection)) / len(union))

In [ ]:
ICD10_jacc = np.zeros((ICD10_idx.shape[0], ICD10_idx.shape[0]))

for i, ICD10_code1 in enumerate(ICD10_idx.ICD10_L):
    for j, ICD10_code2 in enumerate(ICD10_idx.ICD10_L):
        ICD10_jacc[i,j] = Jaccard_Similarity(ICD10_code1, ICD10_code2)

In [ ]:
# save(K_drug_smile_cos, "../Final_DF/DCDB_drug_smile_cos.txt")
# save(K_drug_smile_tanimoto, "../Final_DF/DCDB_drug_smile_tanimoto.txt")
# save(K_drug_morgan_cos, "../Final_DF/DCDB_drug_morgan_cos.txt")
# save(K_drug_morgan_tanimoto, "../Final_DF/DCDB_drug_morgan_tanimoto.txt")

# save(K_ICD10_pair_cos, "../Final_DF/DCDB_ICD10_pair_cos.txt")
# save(K_ICD10_pair_jacc, "../Final_DF/DCDB_ICD10_pair_jacc.txt")
# save(ICD10_hier_sim, "../Final_DF/DCDB_ICD10_hier.txt")
# save(ICD10_jacc, "../Final_DF/DCDB_ICD10_jacc.txt")